In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Cleaning

In [14]:
vals = np.array(["4", 5, "2", 2.3, "3.12"]) 
df = pd.DataFrame(vals)
df.at[0, 0] * 4 # this is messed up because it's a string!

'4444'

In [15]:
cleaned = pd.DataFrame(df[df.columns[0]].apply(lambda x: float(x)))
cleaned

,0
0,4.00
1,5.00
2,2.00
3,2.30
4,3.12


In [16]:
cleaned.at[0, 0] * 4 # works now

16.0

# Imputation

You will often encounter missing data in your datasets. Imputation is a fancy word for dealing with missing values.

Take, for example, this dataframe with several NaN and None values:

In [17]:
vals = {'hack': [1, np.nan, 3, None, 4, np.nan, np.nan], 'cville': ['a', np.nan, 'c', 'd', 'e', 'f', 'g']}
df = pd.DataFrame(vals)
df

,hack,cville
0,1.0,a
1,NaN,NaN
2,3.0,c
3,NaN,d
4,4.0,e
5,NaN,f
6,NaN,g


This will cause problems with many pandas and matplotlib methods.

### 1. Dropping rows with missing values

By far the easiest way of dealing with missing data is just dropping rows that have missing values:

In [18]:
dropped = df.dropna()
dropped

,hack,cville
0,1.0,a
2,3.0,c
4,4.0,e


You might also need to drop NAs only for a single column. In this case, we can pass an argument to .dropna()

In [19]:
df.dropna(subset=['cville']) # if you had column names, subset could be a list of column names

,hack,cville
0,1.0,a
2,3.0,c
3,NaN,d
4,4.0,e
5,NaN,f
6,NaN,g


 ### 2. Filling in missing values

We can fill in missing values in a variety of different ways. You can replace misisng values with a specific value (like the mean), forward-fill, back-fill, or use a variety of more advanced imputation methods such as KNN.

- Replacing missing values with a specific value:

In [20]:
df.fillna(0) # replace missing values with 0

,hack,cville
0,1.0,a
1,0.0,0
2,3.0,c
3,0.0,d
4,4.0,e
5,0.0,f
6,0.0,g


You can also use this to replace missing values with the mean of the values:

In [21]:
df['hack'].fillna(df['hack'].mean())

0    1.000000
1    2.666667
2    3.000000
3    2.666667
4    4.000000
5    2.666667
6    2.666667
Name: hack, dtype: float64

In [22]:
df.fillna({'hack': df['hack'].mean(), 'cville': 'b'})

,hack,cville
0,1.000000,a
1,2.666667,b
2,3.000000,c
3,2.666667,d
4,4.000000,e
5,2.666667,f
6,2.666667,g


- another filling strategy that can be useful for timeseries data is back-fill and forward-fill in which we replace missing values with either the first value before or after the missing field.

In [23]:
df.fillna(method='ffill') # forward-fill

,hack,cville
0,1.0,a
1,1.0,a
2,3.0,c
3,3.0,d
4,4.0,e
5,4.0,f
6,4.0,g


In [24]:
df.fillna(method='bfill') # back-fill
# oh no! the nan at the back did not get filled. this is the issue with ffill and bfill

,hack,cville
0,1.0,a
1,3.0,c
2,3.0,c
3,4.0,d
4,4.0,e
5,NaN,f
6,NaN,g


## Merging DataFrames

Sometimes you have data from two different sources that you'd like to have in one data frame to analyze. We can do that with .concat() and .merge().

In [25]:
left = pd.DataFrame({'Dining': ['Castle', 'Newcomb', 'Chick-fil-A', 'Five Guys', 'Runk', 'Subway'], 'Rating': [4, 3, 5, 4, 4, 3]})
left

,Dining,Rating
0,Castle,4
1,Newcomb,3
2,Chick-fil-A,5
3,Five Guys,4
4,Runk,4
5,Subway,3


In [26]:
right = pd.DataFrame({'Dining': ['Castle', 'Newcomb', 'Chick-fil-A', 'Five Guys', 'Runk', 'Subway'], 'Price': [8.5, 9.5, 6.5, 7.75, 9.5, 6]})
right

,Dining,Price
0,Castle,8.50
1,Newcomb,9.50
2,Chick-fil-A,6.50
3,Five Guys,7.75
4,Runk,9.50
5,Subway,6.00


Now we want to put these into one data table, but not duplicate the "Dining" column. You can call .merge() on two data tables and specify the column to merge "on="

In [27]:
pd.merge(left, right, on='Dining')

,Dining,Rating,Price
0,Castle,4,8.50
1,Newcomb,3,9.50
2,Chick-fil-A,5,6.50
3,Five Guys,4,7.75
4,Runk,4,9.50
5,Subway,3,6.00


Let's try .concat() to see if it does the same thing. It's a little different from .merge() because you have to pass in a list of dataframes that you want to concatenate.

In [28]:
pd.concat([left, right])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Dining,Price,Rating
0,Castle,NaN,4.0
1,Newcomb,NaN,3.0
2,Chick-fil-A,NaN,5.0
3,Five Guys,NaN,4.0
4,Runk,NaN,4.0
5,Subway,NaN,3.0
0,Castle,8.50,NaN
1,Newcomb,9.50,NaN
2,Chick-fil-A,6.50,NaN
3,Five Guys,7.75,NaN


So that didn't really work, because now we have duplicate rows with missing information. However, pd.concat() can be useful for a different case.

In [29]:
more_data = pd.DataFrame({'Dining': ["O'Hill", "Starbucks", "N2Go", "Burrito Theory"], 'Rating': [3, 4, 4, 3]})
more_data

,Dining,Rating
0,O'Hill,3
1,Starbucks,4
2,N2Go,4
3,Burrito Theory,3


In [30]:
pd.concat([left, more_data])

,Dining,Rating
0,Castle,4
1,Newcomb,3
2,Chick-fil-A,5
3,Five Guys,4
4,Runk,4
5,Subway,3
0,O'Hill,3
1,Starbucks,4
2,N2Go,4
3,Burrito Theory,3


Instead of adding rows below the current table, `concat` can actually add columns. Use the axis=1 argument. Here's a case where it might be useful.

In [31]:
more_info = pd.DataFrame({'Popularity': [8, 5, 10, 7, 8, 7], 'Hours': ["7:00-9:00", "7:00-8:00", "11:00-8:00", "11:00-8:00", "7:00-8:00", "11:00-8:00"]})
more_info

,Popularity,Hours
0,8,7:00-9:00
1,5,7:00-8:00
2,10,11:00-8:00
3,7,11:00-8:00
4,8,7:00-8:00
5,7,11:00-8:00


In [32]:
df = pd.concat([right, more_info], axis=1) 
df

,Dining,Price,Popularity,Hours
0,Castle,8.50,8,7:00-9:00
1,Newcomb,9.50,5,7:00-8:00
2,Chick-fil-A,6.50,10,11:00-8:00
3,Five Guys,7.75,7,11:00-8:00
4,Runk,9.50,8,7:00-8:00
5,Subway,6.00,7,11:00-8:00


Note the difference between .concat(axis=1) and .merge(). We would use .concat() when there isn't a duplicate column, and .merge() when there is one.

### A brief aside: cleaning time series data

Let's say UVA decided to close all dining halls one hour later. We could go in and manually fix the hours, but that's not efficient if there were more than 10 dining halls in the table. Instead, we can extract the times we need and turn them into timestamps, which can be easy to add and subtract minutes, hours, etc.

We need to split the hours into open and close time first.

In [33]:
df['Open'] = df['Hours'].apply(lambda x: x.split('-')[0])
df['Close'] = df['Hours'].apply(lambda x: x.split('-')[1])
df

,Dining,Price,Popularity,Hours,Open,Close
0,Castle,8.50,8,7:00-9:00,7:00,9:00
1,Newcomb,9.50,5,7:00-8:00,7:00,8:00
2,Chick-fil-A,6.50,10,11:00-8:00,11:00,8:00
3,Five Guys,7.75,7,11:00-8:00,11:00,8:00
4,Runk,9.50,8,7:00-8:00,7:00,8:00
5,Subway,6.00,7,11:00-8:00,11:00,8:00


In [34]:
df['Open'] = pd.to_datetime(df['Open'])
df['Close'] = pd.to_datetime(df['Close'])
df

,Dining,Price,Popularity,Hours,Open,Close
0,Castle,8.50,8,7:00-9:00,2019-09-22 07:00:00,2019-09-22 09:00:00
1,Newcomb,9.50,5,7:00-8:00,2019-09-22 07:00:00,2019-09-22 08:00:00
2,Chick-fil-A,6.50,10,11:00-8:00,2019-09-22 11:00:00,2019-09-22 08:00:00
3,Five Guys,7.75,7,11:00-8:00,2019-09-22 11:00:00,2019-09-22 08:00:00
4,Runk,9.50,8,7:00-8:00,2019-09-22 07:00:00,2019-09-22 08:00:00
5,Subway,6.00,7,11:00-8:00,2019-09-22 11:00:00,2019-09-22 08:00:00


So the close hours are actually incorrect because datetime uses 24 hour time notation. So we need to add 12 hours to it, plus the 1 hour for UVA closing the dining halls later

In [35]:
df['Close'] = df['Close'] + pd.Timedelta(hours=13)
df

,Dining,Price,Popularity,Hours,Open,Close
0,Castle,8.50,8,7:00-9:00,2019-09-22 07:00:00,2019-09-22 22:00:00
1,Newcomb,9.50,5,7:00-8:00,2019-09-22 07:00:00,2019-09-22 21:00:00
2,Chick-fil-A,6.50,10,11:00-8:00,2019-09-22 11:00:00,2019-09-22 21:00:00
3,Five Guys,7.75,7,11:00-8:00,2019-09-22 11:00:00,2019-09-22 21:00:00
4,Runk,9.50,8,7:00-8:00,2019-09-22 07:00:00,2019-09-22 21:00:00
5,Subway,6.00,7,11:00-8:00,2019-09-22 11:00:00,2019-09-22 21:00:00


(We actually can't isolate only the hour without converting it back to an int)

#Data Visualization

see kaggle notebooks in the 9/30 slides

SNS pairplot: https://seaborn.pydata.org/generated/seaborn.pairplot.html



In [36]:
from google.colab import files
files.upload()

{}

In [37]:
import pandas as pd
df = pd.read_csv('titanic.csv', sep='\t')

FileNotFoundError: ignored

In [0]:
df.head()